In [5]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [122]:
%%run_python #%%transpile_to_aseba # Change to show the code generated in Aseba

#IMPORTANT:
#this programm needs to be launched only when the obsatcle is already detected
#i.e. the sensors already saw something
#otherwise it fucks everything up

# This program implements obstacle avoidance using an ANN
# The inputs x1-7 are taken from the proximity sensors and scaled
# The inputs are multiplied by the weights 
#   and used to set the motor powers
# The center button starts and stops the robot

#sates:
#0 the robot stops 
#1 the robot arrives at the obstacle and choses which side to go around goes forward until the sensor chose (1 left or 5 right) doesnt see anythin

#3 the robot goes forward 1 thymio length
#4 the robot rotates until its sensor (1 or 5) sees the obstacle
#5 the robot follows the obstaclke on its length until its sees nothin
#6 the robot goes forward to completely avoid the obstacle 

# State for start and stop, why is this instruction here? (hint: look at the aseba transpile code)
adjust = 2000
close_thresh = 1
time = 0
state = 1
y = [0,0] 
timer_period[0] = 10
time_one_thym_dist = 150
NONE = 0
side = NONE 
    
@onevent
def button_center():
    global state
    if button_center == 1:
        state = 1 if state==0 else 0

@onevent
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target, button_center, state, y, NONE, side, time
    #side constants
    LEFT = 1
    RIGHT = 2
    
    #asymetric w_l[1] != w_r[1] so the robot choses one side if it arrives straight on the obstacle
    w_l = [10, -15, -10, 6, 0]
    w_r = [-10, -10, 10, 0, 6]

    # Scale factors for sensors and constant factor
    sensor_scale = 400
    
    
    x = [0,0,0,0,0]
    
    left_prox = prox_horizontal[0]// sensor_scale
    right_prox = prox_horizontal[4]// sensor_scale
    
### STATE 0 the robot stops###    
    if state == 0:
        time = 0
        y = [0,0]
        
### STATE 1  the robot arrives at the obstacle and choses which side to go around###         
    if state == 1:
        # Memory
        x[3] = y[0]//10
        x[4] = y[1]//10        
        
        for i in range(1,4): #we only take the 3 sensors of the middle
            # Get and scale inputs
            x[i-1] = prox_horizontal[i] // sensor_scale
            
        y = [60,60] 
        tot = x[0]+x[1]+x[2]
        
        for i in range(len(x)):    
            # Compute outputs of neurons and set motor powers
            y[0] = y[0] + x[i] * w_l[i]
            y[1] = y[1] + x[i] * w_r[i]
        
        if tot < close_thresh :#the three middle sensors see nothing
            if left_prox > close_thresh:
                side = LEFT
            if right_prox > close_thresh:
                side = RIGHT
            if left_prox < close_thresh and right_prox < close_thresh: #the thymio turned and stops only if it sees nothin anymore
                state = 3
    
### STATE 3 and 6 : the robot goes forward 1 thymio length ###                   
    if state == 3 or state == 6:

        time += 1         
        if time <= time_one_thym_dist:
            y = [160,160]
        elif time > time_one_thym_dist:
            y = [0,0]
            if state == 3:
                time = 0
                state = 4
            if state == 6:
                time = 0
                state = 0
                #state = 0 #stops the robot and terminates the avoidance 
            
### STATE 4 the robot rotates until its sensor (1 or 5) sees the obstacle###   
    adj = 5
    if state == 4: 
        if side == RIGHT:
            if right_prox <= close_thresh + adj:
                y = [160,0]
            if right_prox > close_thresh + adj:
                state = 5
        if side == LEFT:
            if left_prox <= close_thresh + adj:
                y = [0,160]
            if left_prox > close_thresh + adj:
                state = 5
                
### STATE 5 the robot follows the obstacle on its length until its sees nothing###             
    if state == 5:
        constant_scale = 20
        left_prox_adjust = left_prox -adjust//sensor_scale
        right_prox_adjust = right_prox -adjust//sensor_scale
        if side == RIGHT:
            y = [80,80+right_prox_adjust*constant_scale]
        if side == LEFT:
            y = [80+left_prox_adjust*constant_scale,80]
        if left_prox < close_thresh and right_prox < close_thresh:
            state = 6
        

    # Set motor powers
    motor_left_target = y[0]
    motor_right_target = y[1]
                
            

            